In [387]:
from pulp import *
import pulp
import pandas as pd
import numpy as np
import random

In [388]:
# Read the Excel file into a DataFrame
df = pd.read_excel('file.xlsx')

# Convert the DataFrame to a NumPy array
data = df.values
products = []

for row in data:
    predictedProfits = row[0]
    predictedProductQuantities = row[1]
    dimension = row[2]
    productName = row[3]

    product = {'predictedProfits': predictedProfits, 'predictedProductQuantities': predictedProductQuantities, 'dimension': dimension, 'productName': productName}
    products.append(product)

print(products)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1'}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2'}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3'}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4'}]


In [389]:
def calculate_profit(products):
    for product in products:
        profit = product['predictedProfits'] / product['predictedProductQuantities']
        product['productProfit'] = profit
    return products

In [390]:
productTest = calculate_profit(products)
print(productTest)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]


In [391]:
# Creation of the ILP model 
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

In [392]:
# Define the decision variables
decisionVariables = []
def createDictionaryWithDecisionVariable():
    for product in products:
        decisionVariable = {product['productName']:  pulp.LpVariable(product['productName'], lowBound=0, cat='Integer')}
        decisionVariables.append(decisionVariable)
        
createDictionaryWithDecisionVariable()
print(decisionVariables)

[{'p1': p1}, {'p2': p2}, {'p3': p3}, {'p4': p4}]


In [393]:
# function to calculate objective function

def objectiveFunction(products):
    model = 0
    count = 0
    for product in products:
        model = model + (product['predictedProfits'] * decisionVariables[count][product['productName']])
        count = count + 1
    return model

#objective function
model += objectiveFunction(products)
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [394]:
#add user input for the multiple variables

minTotalAmount = int(input("Please insert minimum total amount of products: "))
maxTotalAmount = int(input("Please insert maximum  total amount of products: "))

#create two list with min and max amount per product by user insert

minAmountProduct = int(input("Please insert minimum  amount of products : "))
maxAmountProduct  = int(input("Please insert maximum  amount of products : "))

minProductVariety = int(input("Please insert minimum number of product variety: "))
maxProductVariety  = int(input("Please insert maximum  number of product variety: "))

storeDimension  = int(input("Please insert store dimension: "))

In [395]:
# Define the constraints

 # function to calculate variety of products
def get_variety(products):
    # Create an empty set to store the product names
    product_set = []
    
    # Iterate over the products
    for product in products:
        # Get the product name and add it to the set
        product_name = product['productName']
        product_set.append(product_name)
        
    # Return the length of the set
    return len(product_set)

print(get_variety(products))
variety=get_variety(products)

#see why it results in Infeasible
# model += variety <= maxProductVariety
# model += variety >= minProductVariety


#restriction to total amount of products

def maxMinTotalProductsRestriction(products):
    restrictFunction = 0
    count = 0
    for product in products:
        restrictFunction = restrictFunction + decisionVariables[count][product['productName']]
        count = count + 1
    return restrictFunction
    
model += maxMinTotalProductsRestriction(products) <= maxTotalAmount #restricao de quantidade maximo total de produtos
model += maxMinTotalProductsRestriction(products) >= minTotalAmount #restricao de valor minimo total de produtos

# variety of products

#restriction to maximum quantity of products
def maxMinQuantityProductsRestriction(products,model):
    count = 0
    for product in products:
        model += decisionVariables[count][product['productName']] <= maxAmountProduct
        model += decisionVariables[count][product['productName']] >= minAmountProduct
        count = count + 1
    
maxMinQuantityProductsRestriction(products, model)

print(maxAmountProduct)
print(minAmountProduct)


def maxDimensionProductsRestriction(products):
    count = 0
    restrictFunction = 0
    for product in products:                            
        restrictFunction = restrictFunction + (product['dimension'] * decisionVariables[count][product['productName']])
        count = count +1
    return restrictFunction

print(maxDimensionProductsRestriction(products))
#restricao de dimensões de produtos, menor que dimensao todal da loja
model += maxDimensionProductsRestriction(products) <= storeDimension 


4
10
1
5*p1 + 2*p2 + p3 + 20*p4


In [396]:
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
SUBJECT TO
_C1: p1 + p2 + p3 + p4 <= 100

_C2: p1 + p2 + p3 + p4 >= 10

_C3: p1 <= 10

_C4: p1 >= 1

_C5: p2 <= 10

_C6: p2 >= 1

_C7: p3 <= 10

_C8: p3 >= 1

_C9: p4 <= 10

_C10: p4 >= 1

_C11: 5 p1 + 2 p2 + p3 + 20 p4 <= 1.11111111111e+27

VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [397]:
# Solve the problem
status = model.solve()

In [398]:
print(pulp.LpStatus[status])

Optimal


In [399]:
#print(pulp.value(p1), pulp.value(model.objective))

In [400]:
# Print the solution
#print("Solution:")
#print("x =", x.value())
#print("y =", y.value())

In [401]:
def printValuesReturned():
    for v in model.variables():
        try:
            print(v.name,"=", v.value())
        except:
            print("error couldnt find value")
            
printValuesReturned()
print(len(products))
print(products)
print(products[0]['dimension'])

p1 = 10
p2 = 10
p3 = 10
p4 = 10
4
[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]
5


In [402]:
# Genetic Algorithm

productsList = []
print(productsList)
 
for product in products:
    productsList.append((product['productName'], product['predictedProductQuantities'], product['predictedProfits'], product['dimension']))
for p in productsList:
    print(p)

[]
('p1', 10, 100, 5)
('p2', 20, 200, 2)
('p3', 30, 300, 1)
('p4', 1, 10, 20)


In [403]:
# Functions to validate restrictions
# return 1 if restriction is repected else 0

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)
    if (somaTotalAmountIndividuo >= minTotalAmount) and (somaTotalAmountIndividuo <= maxTotalAmount):
        return 1
    else:
        return 0
    
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

#Dimension Restriction
def validateDimension(individuo, products):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimension):
        return 0
    else:
        return 1

In [404]:
# Create First Population

population = []
populationSize=100

def firstpopulation(products):

    while len(population) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(products)):
            
            quantity = random.randint(0, maxAmountProduct)
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution, products)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            population.append(solution)
            
    return population

In [405]:
firstpopulation(products)

print(len(firstpopulation(products)))
print(population)

100
[[7, 3, 8, 2], [5, 5, 7, 7], [4, 7, 4, 6], [4, 1, 3, 9], [10, 7, 4, 1], [5, 6, 1, 2], [4, 5, 6, 5], [5, 7, 5, 5], [1, 9, 6, 7], [8, 8, 3, 5], [5, 9, 10, 7], [8, 7, 1, 1], [10, 7, 7, 4], [10, 7, 5, 1], [7, 5, 6, 1], [2, 2, 10, 9], [9, 6, 8, 9], [3, 4, 9, 8], [8, 8, 6, 1], [8, 4, 3, 6], [3, 8, 2, 4], [3, 10, 10, 1], [9, 3, 6, 9], [8, 3, 6, 7], [5, 7, 10, 2], [2, 9, 8, 8], [9, 5, 8, 7], [3, 3, 8, 10], [1, 9, 8, 8], [8, 1, 7, 8], [6, 1, 5, 7], [3, 4, 6, 9], [3, 10, 9, 10], [10, 6, 1, 5], [10, 8, 2, 8], [3, 5, 4, 1], [5, 10, 3, 7], [10, 2, 7, 4], [7, 10, 5, 5], [3, 6, 7, 4], [2, 7, 10, 1], [4, 5, 6, 8], [10, 1, 7, 8], [1, 5, 6, 3], [1, 6, 5, 8], [1, 3, 3, 9], [8, 7, 6, 5], [9, 10, 10, 9], [6, 1, 7, 2], [1, 9, 8, 5], [9, 2, 2, 10], [5, 6, 4, 10], [6, 8, 7, 1], [7, 6, 3, 3], [1, 9, 9, 8], [2, 5, 7, 9], [1, 4, 7, 7], [7, 1, 7, 6], [6, 2, 5, 8], [3, 6, 6, 4], [1, 7, 5, 8], [4, 7, 8, 4], [8, 3, 3, 1], [8, 7, 10, 9], [6, 7, 2, 2], [9, 9, 1, 1], [8, 2, 6, 9], [6, 1, 7, 5], [10, 2, 1, 4], [4, 1

In [406]:

arraySomaProfits = []
arrayFitProfitAndQuantity = []
finalPopulation = []

# if this element is 1 the first tax of best individuos is added to final population
elementToGetFirstBestIndividuos = 0

taxNoCrossover = 0.10

def createFitnes(population, products):
    
    for person in population:
        
        soma = 0
        contador = 0
        
        for productQuantity in person:
            
            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1
            
        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.append(auxVar)
        arraySomaProfits.append(soma)
        sorted_data = sorted(arrayFitProfitAndQuantity, key=lambda x: x['FitProfit'], reverse=True)
    contador+=1
        
    # array = np.array(arraySomaProfits)
    # array[::-1].sort()
    # print(array)
    print("len",len(sorted_data))
    print("sorted_data",sorted_data)
    return sorted_data
lis=createFitnes(population, products)


len 100
sorted_data [{'FitProfit': 5990, 'FitQuantities': [9, 10, 10, 9]}, {'FitProfit': 5740, 'FitQuantities': [10, 10, 9, 4]}, {'FitProfit': 5470, 'FitQuantities': [4, 10, 10, 7]}, {'FitProfit': 5380, 'FitQuantities': [9, 7, 10, 8]}, {'FitProfit': 5370, 'FitQuantities': [5, 9, 10, 7]}, {'FitProfit': 5310, 'FitQuantities': [3, 10, 10, 1]}, {'FitProfit': 5290, 'FitQuantities': [8, 7, 10, 9]}, {'FitProfit': 5290, 'FitQuantities': [8, 7, 10, 9]}, {'FitProfit': 5100, 'FitQuantities': [3, 10, 9, 10]}, {'FitProfit': 5070, 'FitQuantities': [7, 8, 9, 7]}, {'FitProfit': 5030, 'FitQuantities': [7, 8, 9, 3]}, {'FitProfit': 4920, 'FitQuantities': [5, 7, 10, 2]}, {'FitProfit': 4710, 'FitQuantities': [6, 7, 9, 1]}, {'FitProfit': 4680, 'FitQuantities': [1, 9, 9, 8]}, {'FitProfit': 4630, 'FitQuantities': [10, 3, 10, 3]}, {'FitProfit': 4610, 'FitQuantities': [2, 7, 10, 1]}, {'FitProfit': 4590, 'FitQuantities': [9, 6, 8, 9]}, {'FitProfit': 4540, 'FitQuantities': [10, 7, 7, 4]}, {'FitProfit': 4480, 'Fit

In [407]:
#saves the first best quantity% to the final list
#finalPopulation = []
def getBestXElementsDic(lis, percentage):
   
    for i in range(len(lis)//percentage):
 
        #returns {'FitProfit': 2850, 'FitQuantities': [5, 4, 5, 5]},...
        finalPopulation.append(lis[i])
        #returns  [5, 4, 5, 5],....
        #finalPopulation.append(lis[i]['FitQuantities'])
       
  

    return finalPopulation

dic=getBestXElementsDic(lis,10)
print("-----------------------------------------finalPopulationList---------------------------------------------------")
print(dic)
print("--------------------------------------------------------------------------------------------------------")


-----------------------------------------finalPopulationList---------------------------------------------------
[{'FitProfit': 5990, 'FitQuantities': [9, 10, 10, 9]}, {'FitProfit': 5740, 'FitQuantities': [10, 10, 9, 4]}, {'FitProfit': 5470, 'FitQuantities': [4, 10, 10, 7]}, {'FitProfit': 5380, 'FitQuantities': [9, 7, 10, 8]}, {'FitProfit': 5370, 'FitQuantities': [5, 9, 10, 7]}, {'FitProfit': 5310, 'FitQuantities': [3, 10, 10, 1]}, {'FitProfit': 5290, 'FitQuantities': [8, 7, 10, 9]}, {'FitProfit': 5290, 'FitQuantities': [8, 7, 10, 9]}, {'FitProfit': 5100, 'FitQuantities': [3, 10, 9, 10]}, {'FitProfit': 5070, 'FitQuantities': [7, 8, 9, 7]}]
--------------------------------------------------------------------------------------------------------


In [408]:
#saves the first best quantity% to the final list
#finalPopulation = []
def getBestXElementsQuantity(lis, percentage):
   
    for i in range(len(lis)//percentage):
 
        #returns {'FitProfit': 2850, 'FitQuantities': [5, 4, 5, 5]},...
        #finalPopulation.append(lis[i])
        #returns  [5, 4, 5, 5],....
        finalPopulation.append(lis[i]['FitQuantities'])
       
  

    return finalPopulation

quantities=getBestXElementsQuantity(lis,10)
print("-----------------------------------------finalPopulationList---------------------------------------------------")
print(quantities)
print("--------------------------------------------------------------------------------------------------------")


-----------------------------------------finalPopulationList---------------------------------------------------
[{'FitProfit': 5990, 'FitQuantities': [9, 10, 10, 9]}, {'FitProfit': 5740, 'FitQuantities': [10, 10, 9, 4]}, {'FitProfit': 5470, 'FitQuantities': [4, 10, 10, 7]}, {'FitProfit': 5380, 'FitQuantities': [9, 7, 10, 8]}, {'FitProfit': 5370, 'FitQuantities': [5, 9, 10, 7]}, {'FitProfit': 5310, 'FitQuantities': [3, 10, 10, 1]}, {'FitProfit': 5290, 'FitQuantities': [8, 7, 10, 9]}, {'FitProfit': 5290, 'FitQuantities': [8, 7, 10, 9]}, {'FitProfit': 5100, 'FitQuantities': [3, 10, 9, 10]}, {'FitProfit': 5070, 'FitQuantities': [7, 8, 9, 7]}, [9, 10, 10, 9], [10, 10, 9, 4], [4, 10, 10, 7], [9, 7, 10, 8], [5, 9, 10, 7], [3, 10, 10, 1], [8, 7, 10, 9], [8, 7, 10, 9], [3, 10, 9, 10], [7, 8, 9, 7]]
--------------------------------------------------------------------------------------------------------


In [409]:
#removes previously saved elements, and gets the list of elements that are going to crossover
def getRemainingForCrossover(lis, percentage):
    crossoverElements = lis[len(lis)//percentage:]
    return crossoverElements

crossoverElements=getRemainingForCrossover(lis,10)
print("--------------------------------------CrossoverList------------------------------------------------------")
print(crossoverElements)
print("--------------------------------------------------------------------------------------------")

###!!!!!!!!!!!!!!!!! return just quantities so that the next method doesn't need to be called

--------------------------------------CrossoverList------------------------------------------------------
[{'FitProfit': 5030, 'FitQuantities': [7, 8, 9, 3]}, {'FitProfit': 4920, 'FitQuantities': [5, 7, 10, 2]}, {'FitProfit': 4710, 'FitQuantities': [6, 7, 9, 1]}, {'FitProfit': 4680, 'FitQuantities': [1, 9, 9, 8]}, {'FitProfit': 4630, 'FitQuantities': [10, 3, 10, 3]}, {'FitProfit': 4610, 'FitQuantities': [2, 7, 10, 1]}, {'FitProfit': 4590, 'FitQuantities': [9, 6, 8, 9]}, {'FitProfit': 4540, 'FitQuantities': [10, 7, 7, 4]}, {'FitProfit': 4480, 'FitQuantities': [2, 9, 8, 8]}, {'FitProfit': 4470, 'FitQuantities': [3, 10, 7, 7]}, {'FitProfit': 4380, 'FitQuantities': [1, 9, 8, 8]}, {'FitProfit': 4370, 'FitQuantities': [9, 5, 8, 7]}, {'FitProfit': 4360, 'FitQuantities': [10, 6, 7, 6]}, {'FitProfit': 4350, 'FitQuantities': [1, 9, 8, 5]}, {'FitProfit': 4330, 'FitQuantities': [10, 6, 7, 3]}, {'FitProfit': 4310, 'FitQuantities': [6, 8, 7, 1]}, {'FitProfit': 4250, 'FitQuantities': [7, 10, 5, 5]}, 

In [410]:
def getQuantities(crossoverElements):
    
    print(len(crossoverElements))
    fit_quantities = [d['FitQuantities'] for d in crossoverElements]
    return fit_quantities

crossoverElementsQuantities=getQuantities(crossoverElements)

print(crossoverElementsQuantities)

90
[[7, 8, 9, 3], [5, 7, 10, 2], [6, 7, 9, 1], [1, 9, 9, 8], [10, 3, 10, 3], [2, 7, 10, 1], [9, 6, 8, 9], [10, 7, 7, 4], [2, 9, 8, 8], [3, 10, 7, 7], [1, 9, 8, 8], [9, 5, 8, 7], [10, 6, 7, 6], [1, 9, 8, 5], [10, 6, 7, 3], [6, 8, 7, 1], [7, 10, 5, 5], [4, 7, 8, 4], [7, 10, 5, 4], [8, 8, 6, 1], [8, 7, 6, 5], [9, 6, 6, 6], [10, 7, 5, 1], [3, 4, 9, 8], [7, 2, 9, 7], [1, 9, 6, 7], [7, 3, 8, 2], [8, 1, 9, 2], [2, 2, 10, 9], [5, 5, 7, 7], [3, 6, 7, 4], [10, 7, 4, 1], [10, 2, 7, 4], [7, 5, 6, 1], [5, 10, 3, 7], [5, 7, 5, 5], [3, 3, 8, 10], [9, 3, 6, 9], [2, 5, 7, 9], [10, 1, 7, 8], [8, 8, 3, 5], [3, 6, 6, 4], [10, 4, 5, 2], [10, 8, 2, 8], [4, 5, 6, 8], [8, 3, 6, 7], [4, 5, 6, 5], [8, 1, 7, 8], [8, 2, 6, 9], [1, 7, 5, 8], [1, 4, 7, 7], [4, 7, 4, 6], [7, 1, 7, 6], [9, 9, 1, 1], [5, 6, 4, 10], [3, 4, 6, 9], [6, 1, 7, 5], [1, 5, 6, 3], [6, 1, 7, 2], [8, 6, 3, 1], [2, 1, 8, 9], [1, 6, 5, 8], [3, 8, 3, 7], [7, 6, 3, 3], [3, 1, 7, 10], [9, 7, 1, 9], [6, 1, 6, 5], [6, 7, 2, 2], [8, 7, 1, 9], [6, 2, 5,

In [411]:

def crossoverMethod(parent1, parent2, crossover_rate):
    if random.uniform(0, 1) < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    else:
        return parent1, parent2



In [412]:
crossover_rate = 0.8 # Crossover rate
percentageCrossover = 10 # Crossover rate
def crossOver(dic):
    listCrossOver=[]
    for gene in dic:
      
        #selects first and last genes to crossover
        parent1 = dic[0]['FitQuantities']
        parent2 = dic[-1]['FitQuantities']

        child1, child2 = crossoverMethod(parent1, parent2, crossover_rate)
        # listCrossOver.append(child1)
        # listCrossOver.append(child2)
        if child1 not in listCrossOver:
            listCrossOver.append(child1)
        if child2 not in listCrossOver:
            listCrossOver.append(child1)
      


    #return first 10% of the genes resulting from the crossover      !!!! deveriam ser os 10% MELHORES-necessário fazer calculo
    #return listCrossOver[:len(listCrossOver)//percentageCrossover]
    return listCrossOver

print(crossOver(crossoverElements))
print(len(crossOver(crossoverElements)))


[[7, 8, 1, 8], [2, 2, 9, 3], [7, 8, 9, 3], [2, 2, 1, 8], [7, 8, 9, 8], [2, 2, 1, 3], [7, 8, 9, 3], [2, 2, 1, 8], [7, 8, 9, 8], [2, 2, 1, 3], [7, 8, 9, 3], [2, 2, 1, 8], [7, 8, 9, 8], [2, 2, 1, 3], [7, 2, 1, 8], [2, 8, 9, 3], [7, 8, 9, 3], [2, 2, 1, 8], [7, 8, 9, 3], [2, 2, 1, 8], [7, 8, 1, 8], [2, 2, 9, 3], [7, 8, 9, 8], [2, 2, 1, 3], [7, 2, 1, 8], [2, 8, 9, 3], [7, 8, 1, 8], [2, 2, 9, 3], [7, 8, 9, 8], [2, 2, 1, 3], [7, 2, 1, 8], [2, 8, 9, 3], [7, 8, 9, 8], [2, 2, 1, 3], [7, 8, 9, 8], [2, 2, 1, 3], [7, 8, 9, 3], [2, 2, 1, 8], [7, 2, 1, 8], [2, 8, 9, 3], [7, 2, 1, 8], [2, 8, 9, 3], [7, 8, 1, 8], [2, 2, 9, 3], [7, 8, 1, 8], [2, 2, 9, 3], [7, 8, 9, 8], [2, 2, 1, 3], [7, 8, 9, 8], [2, 2, 1, 3], [7, 8, 1, 8], [2, 2, 9, 3], [7, 2, 1, 8], [2, 8, 9, 3], [7, 2, 1, 8], [2, 8, 9, 3], [7, 8, 9, 8], [2, 2, 1, 3], [7, 2, 1, 8], [2, 8, 9, 3], [7, 8, 1, 8], [2, 2, 9, 3], [7, 8, 9, 3], [2, 2, 1, 8], [7, 8, 9, 8], [2, 2, 1, 3], [7, 8, 1, 8], [2, 2, 9, 3], [7, 8, 9, 3], [2, 2, 1, 8], [7, 8, 9, 8], [2, 2

In [72]:
# Create Fitness function to give the best solution

arraySomaProfits = []
arrayFitProfitAndQuantity = []
finalPopulation = []

# if this element is 1 the first tax of best individuos is added to final population
elementToGetFirstBestIndividuos = 0

taxNoCrossover = 0.10

def createFitnessFunction(population, products):
    
    for person in population:
        
        soma = 0
        contador = 0
        
        for productQuantity in person:
            
            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1
            
        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.append(auxVar)
        arraySomaProfits.append(soma)
        
    array = np.array(arraySomaProfits)
    array[::-1].sort()
    
    # get out taxNoCrossover to final population
    
    numberElementsNoCrossover = len(population) * taxNoCrossover
    
    for i in range(round(numberElementsNoCrossover)):
        
        for element in arrayFitProfitAndQuantity:
            
            if element['FitProfit'] == array[i]:
                finalPopulation.append(element['FitQuantities'])
                population.remove(element['FitQuantities'])
            
    return array

fitSolutions = createFitnessFunction(population, products)
maxFitSolution = np.max(createFitnessFunction(population, products))

print(fitSolutions)
print(maxFitSolution)
print(finalPopulation)
print(len(population) )

ValueError: list.remove(x): x not in list

In [47]:
print(population[1])
print(random.randint(0, len(population[0])))

[5, 1, 4, 3]
0


In [161]:
# Function to crossover

def crossover(finalPopulation):
    
 #   while len(population) > 1:
        
        firstPai = population[0]
        secondPai = population[1]
        print(firstPai)
        print(secondPai)
        crossoverPoint = random.randint(0, len(population[0]))
        for i in range(crossoverPoint):
            aux = firstPai[i]
            print(aux)
            firstPai[i] = secondPai[i]
            print(firstPai[i])
            secondPai[i] = aux
            print(secondPai[i])
            
        print(firstPai)
        print(secondPai)
 
crossover(population)

[3, 2, 2, 5]
[2, 4, 2, 4]
[3, 2, 2, 5]
[2, 4, 2, 4]


In [222]:
import random


def optimize_product_quantities(productsList, maxTotalQuantity, maxProductQuantity, storeDimension):
    # products is a list of tuples, where each tuple contains the following information about a product:
    # (name, quantity, price, dimension)

    # Create a population of random solutions

    # population = []
    # for i in range(100):
    #     solution = []
    #     totalQuantity = 0
    #     totalDimension = 0
    #     for j in range(len(productsList)):
    #         quantity = random.randint(0, maxProductQuantity)
    #         totalQuantity += quantity
    #         totalDimension += quantity * productsList[j][2]
    #         solution.append(quantity)
    #     if totalQuantity <= maxTotalQuantity and totalDimension <= storeDimension:
    #         population.append(solution)

    #represents code above
    firstpopulation(productsList)

    # Set the number of generations and the mutation rate
    numGenerations = 1000
    mutationRate = 0.01

    # Run the genetic algorithm
    for generation in range(numGenerations):
        # Calculate the fitness of each solution in the population
        populationFitness = []
        for i in range(len(population)):
            totalProfit = 0
            totalQuantity = 0
            totalDimension = 0
            for j in range(len(productsList)):
                totalProfit += population[i][j] * productsList[j][2]
                totalQuantity += population[i][j]
                totalDimension += population[i][j] * productsList[j][3]
            if totalQuantity <= maxTotalQuantity and totalDimension <= storeDimension:
                populationFitness.append(totalProfit)
            else:
                populationFitness.append(0)

        # Select the fittest solutions for breeding
        fittestSolutions = []
        for i in range(int(len(population) / 2)):
            fittestIndex = populationFitness.index(max(populationFitness))
            fittestSolutions.append(population[fittestIndex])
            populationFitness[fittestIndex] = 0
        population = fittestSolutions

        # Breed the solutions to create a new population
        newPopulation = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[random.randint(0, len(population) - 1)]
            child = []
            for j in range(len(productsList)):
                if random.random() < 0.5:
                    child.append(parent1[j])
                else:
                    child.append(parent2[j])

            # Mutate the child
            if random.random() < mutationRate:
                mutationIndex = random.randint(0, len(productsList) - 1)
                child[mutationIndex] = random.randint(0, maxProductQuantity)

            newPopulation.append(child)
        population = newPopulation
        print(generation)
    # Return the fittest solution

    return population[0]

In [223]:
optimize_product_quantities(productsList,maxTotalAmount,maxAmountProduct,storeDimension)

UnboundLocalError: local variable 'population' referenced before assignment

In [185]:


# Define the fitness function
def fitness(x):
    # x is the number of products to buy
    # We can define the fitness as the difference between the desired stock level and the actual stock level
    # The desired stock level is 10
    return abs(10 - x)

# Initialize the population with random solutions
population = [random.randint(0, 20) for _ in range(10)]

# Set the maximum number of generations
max_generations = 100

# Iterate over the generations
for generation in range(max_generations):
    # Evaluate the fitness of each solution
    fitness_values = [fitness(x) for x in population]
    
    # Select the best solutions as parents
    # We'll use tournament selection here, which involves selecting a random subset of the population
    # and choosing the best solution from that subset as the parent
    parent1 = min(random.sample(population, 5), key=fitness)
    parent2 = min(random.sample(population, 5), key=fitness)
    
    # Create offspring by combining the parents using crossover
    # We'll use one-point crossover here, which involves selecting a random point in the parents' genomes
    # and swapping the genes on either side of the point to create the offspring
    crossover_point = random.randint(1, len(parent1))
    offspring = parent1[:crossover_point] + parent2[crossover_point:]
    
    # Add some randomness to the offspring using mutation
    # We'll flip a random bit in the offspring's genome
    mutation_point = random.randint(0, len(offspring) - 1)
    offspring[mutation_point] = 1 - offspring[mutation_point]
    
    # Replace the worst solution in the population with the offspring
    worst_solution = max(zip(population, fitness_values), key=lambda x: x[1])
    population[population.index(worst_solution[0])] = offspring

# The final solution is the best solution in the final generation
best_solution = min(zip(population, fitness_values), key=lambda x: x[1])
print(f"The optimal number of products to buy is: {best_solution[0]}")

TypeError: object of type 'int' has no len()

In [ ]:

  # evaluate the fitness of a given solution
  def evaluate_fitness(products):

#  # check if better (e.g. perform a tournament)
#  if scores[ix] < scores[selection_ix]:
#  selection_ix = ix
#  return pop[selection_ix]
#{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0},  

    bestQuantity = 0
        if products['predictedProfits'] < products['predictedProductQuantities']:
      return 0

    
    total_dimension = sum([q * d for q, d in zip(solution, product_dimensions)])
    if total_quantity < min_quantity or total_quantity > max_quantity or total_dimension > store_dimension:
      return 0
    profit = calculate_profit(solution)
    # calculate the deviation from the target profit
    deviation = abs(profit - target_profit)
    # the fitness is higher for solutions that have a lower deviation from the target profit
    return 1 / (deviation + 1)

  # genetic algorithm function
  def genetic_algorithm(fitness_fn):
    # initialize population
    population = initialize_population(product_quantities)
    for generation in range(max_generations):
      # evaluate fitness
      fitnesses = [fitness_fn(s) for s in population]
      # select parents
      parents = select_parents(population, fitnesses)
      # create offspring
      offspring = create_offspring(parents)
      # mutate offspring
      offspring = mutate(offspring)
      # replace least fit individuals in population with offspring
      population = replace_least_fit(population, offspring)
    # return the fittest solution
    return population[0]

  # use the genetic algorithm to find the optimal solution
  return genetic_algorithm(evaluate_fitness)

In [ ]:
#--------------------------------------------------------------------------
